# Lab 7: Deploy Warehouse Assistant Agent to Amazon Bedrock AgentCore with SAP GenAI Hub

Welcome! In this tutorial, you'll learn how to deploy the intelligent warehouse operations agent from Lab 6 to Amazon Bedrock AgentCore Runtime while using SAP GenAI Hub for token consumption instead of direct Amazon Bedrock models.

## Business Scenario: Scalable Cloud Deployment

**Challenge:** After successfully building and testing our warehouse operations agent locally in Lab 6, we need to deploy it to a production-ready, scalable cloud environment where multiple users can access it simultaneously.

**Solution:** Deploy the agent to Amazon Bedrock AgentCore Runtime while maintaining SAP GenAI Hub integration for:
- Consistent token consumption through SAP's ecosystem
- Centralized governance and compliance
- Cost tracking through SAP's billing structure
- Enterprise-grade security and access controls

## Prerequisites

1. Completed Lab 6 - Warehouse operations agent
2. SAP AI Core credentials in your `~/.aicore/config.json` file
3. AWS credentials configured for your account
4. Amazon Bedrock AgentCore SDK installed
5. Strands Agents SDK installed
6. SAP GenAI Hub SDK installed

## Architecture Overview

In this lab, we'll deploy our multi-agent warehouse system to AgentCore while routing all LLM calls through SAP GenAI Hub:

```
┌─────────────────────────────────────────────────────────────┐
│                Amazon Bedrock AgentCore                      │
│  ┌────────────────────────────────────────────────────────┐ │
│  │              Docker Container                          │ │
│  │  ┌──────────────────────────────────────────────────┐ │ │
│  │  │  Warehouse Agent (from Lab 6)                    │ │ │
│  │  │  - Main orchestrator                             │ │ │
│  │  │  - Selector sub-agent                            │ │ │
│  │  │  - OData caller tool                             │ │ │
│  │  │  - ~/.aicore/config.json                         │ │ │
│  │  └──────────────────────────────────────────────────┘ │ │
│  └────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────┘
                            │
                            │ HTTPS (via SAP GenAI Hub SDK)
                            ▼
┌─────────────────────────────────────────────────────────────┐
│                SAP AI Core / GenAI Hub                       │
│  ┌────────────────────────────────────────────────────────┐ │
│  │  Models Available:                                     │ │
│  │  - Amazon Nova (Pro, Lite, Micro)                     │ │
│  │  - Anthropic Claude (4.5 Sonnet, 4 Sonnet, 3 Haiku)  │ │
│  │  - Amazon Titan Models                                │ │
│  └────────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────┘
```

## Step 1: Install Required Dependencies

Install all required packages from pyproject.toml.

In [ ]:
# Install all dependencies from pyproject.toml
# %pip install .

## Step 2: Create the Agent Code File

Create a Python file that will run in the AgentCore container with the complete warehouse agent from Lab 6.

In [ ]:
%%writefile warehouse_agent_agentcore.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from util.strands_bedrock_sap_genai_hub import SAPGenAIHubModel
from strands import Agent, tool
import os
from pathlib import Path
import yaml
from util.odata_tool import odata_caller

# Initialize AgentCore app
app = BedrockAgentCoreApp()

# Initialize the SAPGenAIHubModel for AgentCore deployment
model = SAPGenAIHubModel(
    model_id="anthropic--claude-4.5-sonnet",
    # Alternative models:
    # model_id="amazon--nova-pro",
    # model_id="amazon--nova-lite",
)

# Load YAML OpenAPI files from directory
def load_openapi_specs(path="./assets/knowledgebase"):
    specs = []
    for file in Path(path).glob("*.y*ml"):
        with open(file, "r") as f:
            try:
                data = yaml.safe_load(f)
                servers = data.get("servers", [])
                base_urls = []
                for s in servers:
                    url = s.get("url")
                    desc = s.get("description", "")
                    if url:
                        base_urls.append({"url": url, "description": desc})
                summary = {
                    "file": file.name,
                    "title": data.get("info", {}).get("title"),
                    "description": data.get("info", {}).get("description"),
                    "paths": list(data.get("paths", {}).keys()),
                    "base_urls": base_urls or [{"url": "/", "description": "Default"}]
                }
                specs.append(summary)
            except Exception as e:
                print(f"Error parsing {file}: {e}") 
    return specs

def specs_to_prompt_string(specs):
    prompt_parts = []
    for spec in specs:
        base_url_str = ", ".join(
            f"{url_info['url']} ({url_info['description']})" for url_info in spec['base_urls']
        )
        paths_str = ", ".join(spec['paths'])
        part = (
            f"API Spec: {spec['file']}\n"
            f"Title: {spec['title']}\n"
            f"Description: {spec['description']}\n"
            f"Base URLs: {base_url_str}\n"
            f"Endpoints: {paths_str}\n"
        )
        prompt_parts.append(part)
    return "\n---\n".join(prompt_parts)

# Create API specs for selector agent
specs = load_openapi_specs()
specs_prompt_string = specs_to_prompt_string(specs)

SELECTOR_SYSTEM_PROMPT = f"""
You are an API selection subagent.
Given a user query and the following list of OpenAPI specs, each with a title, description, base URLs, and available endpoints:

{specs_prompt_string}

Your task is to identify which API and specific endpoint is most appropriate to fulfill the user query.
Provide clear reasoning for your choice, referencing the API spec details provided.
If no suitable API is found, explain why.

Make sure to reply with the sandbox BASE URL.

"""

# Create selector agent
selector_agent = Agent(
    system_prompt=SELECTOR_SYSTEM_PROMPT,
    model=model
)

@tool
def SelectorAPIAgentAsATool(query: str) -> str:
    """
    This analyzes available OpenAPI specs and selects the most appropriate
    API and endpoint based on user queries.

    Args:
        query: User query describing what they want to accomplish

    Returns:
        Agent response with API selection recommendation if successful, 
        error message if initialization fails
    """
    return selector_agent(query).message

# Define the warehouse agent system prompt
warehouse_agent_prompt = """
You are an expert Warehouse Operations Manager for GlobalTech Manufacturing's Distribution Center (Warehouse 1750). 
You have access to real-time SAP warehouse data through dynamic OData API exploration capabilities.

CORE CAPABILITIES:
1. API Structure Exploration: Dynamically discover available data fields and entities
2. Product Discovery: Find available products without hardcoded assumptions
3. Dynamic Querying: Construct intelligent OData queries based on user needs

APPROACH TO PROBLEM SOLVING:
- You must start by using the SelectorAPIAgentAsATool to help you determine which OData API to call
- Next use the $metadata endpoint that to understand the API that SelectorAPIAgentAsATool provides  
- Use dynamic queries to discover information rather than making assumptions
- Leverage OData filtering, sorting, and selection to get precise answers
- You may need to do this multiple times. Always write what URL have constructed so user is informed.

PRODUCT KNOWLEDGE (can be expanded through discovery):
- WM-AN01: Advanced Sensors (high-precision electronic components)
- WM-AN02: Control Units (critical automation hardware)
- WM-AN03: Power Modules (electrical power management systems)
- WM-AN04: Communication Devices (networking and connectivity hardware)

COMMUNICATION STYLE:
- Be professional but conversational and succinct
- Explain your discovery process when exploring new data
- Provide specific, actionable insights with quantitative data
- Do not use emojis

When users ask questions:
1. First determine what data you need to answer the question
3. Feel free to use the odata_caller tool as many times as needed to get the right information.
3. Construct appropriate OData queries to get the specific information needed
4. Analyze the results and provide comprehensive, intelligent responses

Example for using the odata_caller tool:
```python
        odata_caller(
            base_url="",
            endpoint="WarehouseStockProducts",
            operation="get",
            odata_params={"$filter": "Product eq 'WM-AN02'"},
            auth_type="api_key",
            auth_env_var="SAP_S4HANA_PUBLIC_CLOUD_KEY"
        )
        ```

Focus on SAP S/4 HANA OData endpoints, warehouse management APIs, and supply chain operations.

Available tools:
- odata_caller: Universal OData tool for SAP API interactions with built-in authentication and query parameter support
- auth_token: Use os.getenv("SAP_S4HANA_PUBLIC_CLOUD_KEY") to get the API key
Example below:

# For SAP OData calls, use consistent headers
headers = {
    "APIKey": os.getenv("SAP_S4HANA_PUBLIC_CLOUD_KEY"),
    "Accept": "application/json", OR "application/xml" choose as needed 
    "DataServiceVersion": "2.0"
}

The odata_caller tool handles SAP-specific authentication automatically and provides comprehensive error handling and response formatting.

"""

# Create the warehouse agent
warehouse_agent = Agent(
    model=model,
    tools=[
        SelectorAPIAgentAsATool,
        odata_caller,
    ],
    system_prompt=warehouse_agent_prompt
)

# AgentCore entrypoint function
@app.entrypoint
def warehouse_agent_entrypoint(payload):
    """
    Invoke the warehouse agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = warehouse_agent(user_input)
    return response.message

if __name__ == "__main__":
    app.run()

## Step 3: Create Requirements File

Create the requirements file with AgentCore and SAP GenAI Hub dependencies.

In [ ]:
%%writefile requirements.txt
# AgentCore requirements
strands-agents
strands-agents-tools
uv
boto3
bedrock-agentcore<=0.1.5
bedrock-agentcore-starter-toolkit==0.1.14
# SAP GenAI Hub and warehouse agent dependencies
sap-ai-sdk-gen[all]>=5.5.0
pyyaml
requests
python-dotenv

## Step 4: Create SAP AI Core Configuration File

This is the critical step for enabling SAP GenAI Hub token consumption in AgentCore.

In [ ]:
import json
import os

# Load the existing config from ~/.aicore/config.json
config_path = os.path.expanduser('~/.aicore/config.json')
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    # Write the config file that will be copied into the Docker container
    with open('config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    print("SAP AI Core configuration file created: config.json")
    print("This file will be copied into the AgentCore container for GenAI Hub access.")
else:
    print("Error: ~/.aicore/config.json not found!")
    print("Please ensure you have your SAP AI Core credentials configured.")

## Step 5: Configure AgentCore Runtime Deployment

Configure the AgentCore Runtime deployment.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

# Initialize AWS session and AgentCore runtime
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "warehouse_ops_agent_sap_genai_hub"

print(f"Configuring agent deployment in region: {region}")

# Configure the agent deployment
response = agentcore_runtime.configure(
    entrypoint="warehouse_agent_agentcore.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

print("AgentCore configuration completed!")
print("Generated Dockerfile will be modified next...")
response

## Step 6: Modify the Generated Dockerfile

**Critical Step**: Modify the generated Dockerfile to include the SAP GenAI Hub configuration and ensure all dependencies are available in the container.

This step implements:
1. SAP GenAI Hub configuration using `AICORE_HOME` environment variable (per SAP documentation)
2. Essential dependencies (util/ and assets/ directories)

**Updated**: Uses proper SAP GenAI Hub SDK configuration pattern with `AICORE_HOME`.

In [ ]:
# Read the generated Dockerfile
with open('Dockerfile', 'r') as f:
    dockerfile_content = f.read()

# Find the insertion point (before CMD instruction)
lines = dockerfile_content.split('\n')
cmd_index = -1
for i, line in enumerate(lines):
    if line.strip().startswith('CMD'):
        cmd_index = i
        break

if cmd_index > -1:
    # Insert all required configuration before CMD
    config_lines = [
        "",
        "# Copy util directory (required for SAP GenA Hub model and OData tool)",
        "COPY util/ ./util/",
        "",
        "# Copy assets directory (required for OpenAPI knowledgebase)",
        "COPY assets/ ./assets/",
        "",
        "# Copy the config.json from your local machine",
        "COPY config.json /app/.aicore/config.json",
        "",
        "# Set AICORE_HOME environment variable for SAP GenAI Hub SDK",
        "ENV AICORE_HOME=/app/.aicore",
        ""
    ]
    
    # Insert the new lines before CMD
    modified_lines = lines[:cmd_index] + config_lines + lines[cmd_index:]
    modified_content = '\n'.join(modified_lines)
    
    # Write the modified Dockerfile
    with open('Dockerfile', 'w') as f:
        f.write(modified_content)
    
    print(" Dockerfile successfully modified with all required dependencies!")
    print("\n Added the following for AgentCore containerization:")
    print("  COPY util/ ./util/                    # SAP GenAI Hub model & OData tool")
    print("  COPY assets/ ./assets/                # OpenAPI knowledgebase")
    print("  RUN mkdir -p /app/.aicore             # GenAI Hub credentials directory")
    print("  COPY config.json /app/.aicore/config.json  # GenAI Hub credentials")
    print("  ENV AICORE_HOME=/app/.aicore          # SAP GenAI Hub SDK directory")
    print("\n All dependencies now available in AgentCore container!")
else:
    print(" Could not find CMD instruction in Dockerfile")

## Step 7: Launch Agent to AgentCore Runtime

Deploy the agent to Amazon Bedrock AgentCore Runtime.

In [ ]:
# Launch the agent with SAP API key environment variable
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the SAP S/4HANA Public Cloud API key from environment
sap_api_key = os.getenv("SAP_S4HANA_PUBLIC_CLOUD_KEY")
if not sap_api_key:
    import getpass
    sap_api_key = getpass.getpass("Please enter your SAP_S4HANA_PUBLIC_CLOUD_KEY: ")

launch_result = agentcore_runtime.launch(
    env_vars={
        "SAP_S4HANA_PUBLIC_CLOUD_KEY": sap_api_key
    }
)
print("Deployment initiated with SAP API key configured!")
launch_result

## Step 8: Check AgentCore Runtime Status

Monitor the deployment status until it's ready.

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

print(f"Initial status: {status}")

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)

print(f"\nFinal status: {status}")
status

## Step 9: Invoke AgentCore Runtime

Test the warehouse agent running on AgentCore.

In [ ]:
# Test with a warehouse inventory query
invoke_response = agentcore_runtime.invoke({
    "prompt": "I need a complete overview of our current warehouse inventory. What products do we have and how much of each?"
})
invoke_response

## Step 10: Process Invocation Results

Display the agent response in a formatted way.

In [ ]:
from IPython.display import Markdown, display
import json

def display_agentcore_response(response, title="Agent Response"):
    """
    Display an AgentCore response in a formatted way.
    
    Args:
        response: The response dictionary from agentcore_runtime.invoke()
        title: Optional title to display before the response
    """
    print(f"\n=== {title} ===")
    
    try:
        # The response comes as multiple byte chunks that need to be concatenated
        response_chunks = response['response']
        
        # Concatenate all byte chunks into a single bytes object
        complete_response_bytes = b''.join(response_chunks)
        
        # Decode and parse the JSON
        response_data = json.loads(complete_response_bytes.decode('utf-8'))
        
        # Extract the text content from the response structure
        if 'content' in response_data and response_data['content']:
            text_content = response_data['content'][0]['text']
            display(Markdown(text_content))
        else:
            print("No text content found in response")
            print("Raw response structure:")
            print(json.dumps(response_data, indent=2))
            
    except Exception as e:
        print(f"Error processing response: {e}")
        print("Raw response:")
        print(response)

# Display the current response
display_agentcore_response(invoke_response, "Warehouse Inventory Overview")

## Step 11: Advanced Test Scenarios

Run additional warehouse scenarios from Lab 6.

In [ ]:
# Test scenarios from Lab 6
test_scenarios = [
    "We just received an urgent order from ExampleCorp for 200 units of WM-AN02 Control Units. Can we fulfill this order immediately?",
    "Can we fulfill an order for 100 units of WM-AN01 sensors, 75 units of WM-AN03 power modules, and 50 units of WM-AN04 communication devices?",
    "Analyze our current storage utilization and recommend improvements for warehouse efficiency."
]

print("Running advanced warehouse scenarios...\n")

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n--- Test Scenario {i} ---")
    print(f"Query: {scenario}")
    try:
        response = agentcore_runtime.invoke({"prompt": scenario})
        display_agentcore_response(response, f"Test Scenario {i} Results")
    except Exception as e:
        print(f"❌ Failed: {e}\n")

print("🎉 All test scenarios completed!")

## Step 12: Invoke AgentCore Runtime with boto3 (Optional)

You can also invoke the agent directly using boto3 for integration with other AWS services.

In [ ]:
import boto3

# Get the agent ARN from launch result
agent_arn = launch_result.agent_arn

# Create AgentCore client
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# Invoke using boto3
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is our current inventory for WM-AN02 products?"})
)

# Process the response stream
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        # Convert boto3 response format to match agentcore_runtime.invoke format
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
        
        if events:
            # Create a response format compatible with display_agentcore_response
            formatted_response = {'response': events}
            display_agentcore_response(formatted_response, "Boto3 Response - WM-AN02 Inventory")
        else:
            print("No response events received")
            
    except Exception as e:
        print(f"Error processing boto3 response: {e}")
        print("Raw response:")
        print(boto3_response)

## Step 14: Cleanup (Optional)

Clean up the AgentCore Runtime when no longer needed.

In [ ]:
# Uncomment to delete the AgentCore deployment
# import boto3
# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )
# ecr_client = boto3.client(
#     'ecr',
#     region_name=region
# )

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

## Summary


**Congratulations!** You've successfully deployed an enterprise-ready AI agent that combines Amazon Bedrock AgentCore with SAP GenAI Hub's governed AI consumption model.